In [1]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

In [2]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd

# Read the CSV and Perform Basic Data Cleaning

In [4]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


# Select your features (columns)

In [5]:
# Set features. This will also be used as your x values.
X = df.drop("koi_disposition", axis=1)
y = df["koi_disposition"]
print(X.shape, y.shape)

(6991, 40) (6991,)


# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

# Create a Train Test Split

Use `koi_disposition` for the y values

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler


X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [7]:
y_train.unique()

array(['CANDIDATE', 'FALSE POSITIVE', 'CONFIRMED'], dtype=object)

# Pre-processing

# Train the model

In [8]:
# Scale your data
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [9]:
from sklearn.ensemble import RandomForestClassifier

# Create a random forest classifier
rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X_train_scaled, y_train)
rf.score(X_train_scaled, y_train)

1.0

In [10]:
from sklearn.ensemble import RandomForestClassifier

# Create a random forest classifier
rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X_test_scaled, y_test)
rf.score(X_test_scaled, y_test)

1.0

In [11]:
# Random Forests in sklearn will automatically calculate feature importance
importances = rf.feature_importances_
importances

array([0.08135789, 0.05777772, 0.10560727, 0.0296437 , 0.02034721,
       0.02006924, 0.01604791, 0.01533981, 0.0243607 , 0.02381398,
       0.01711941, 0.01183152, 0.01001223, 0.02656146, 0.02917144,
       0.0340751 , 0.01953808, 0.01568883, 0.0146608 , 0.04843114,
       0.03937568, 0.03546558, 0.01584544, 0.01430527, 0.01578012,
       0.01113462, 0.06986789, 0.00335719, 0.00947125, 0.03348684,
       0.02432831, 0.01310468, 0.01094951, 0.01155416, 0.01022858,
       0.01242207, 0.0108752 , 0.01442088, 0.01222263, 0.01034863])

In [12]:
# We can sort the features by their importance
sorted(zip(rf.feature_importances_, X.columns), reverse=True)

[(0.10560727449230438, 'koi_fpflag_co'),
 (0.08135789130507573, 'koi_fpflag_nt'),
 (0.06986788734616226, 'koi_model_snr'),
 (0.05777771869979719, 'koi_fpflag_ss'),
 (0.048431139638965386, 'koi_prad'),
 (0.03937567861399554, 'koi_prad_err1'),
 (0.0354655833699466, 'koi_prad_err2'),
 (0.034075097513545774, 'koi_duration_err2'),
 (0.033486844850748515, 'koi_steff_err1'),
 (0.029643697230829744, 'koi_fpflag_ec'),
 (0.029171442232125293, 'koi_duration_err1'),
 (0.026561461957057896, 'koi_duration'),
 (0.024360701727265566, 'koi_time0bk_err1'),
 (0.024328305246551146, 'koi_steff_err2'),
 (0.023813980545657146, 'koi_time0bk_err2'),
 (0.020347205439176022, 'koi_period'),
 (0.0200692359330442, 'koi_period_err1'),
 (0.01953808489131889, 'koi_depth'),
 (0.017119406958287037, 'koi_impact'),
 (0.016047914359381955, 'koi_period_err2'),
 (0.015845444810299737, 'koi_teq'),
 (0.01578012490611691, 'koi_insol_err1'),
 (0.015688827669938406, 'koi_depth_err1'),
 (0.015339808451556533, 'koi_time0bk'),
 (0.0

# Hyperparameter Tuning
Use GridSearchCV to tune the model's parameters



In [13]:
# Create the GridSearch estimator along with a parameter object containing the values to adjust
from sklearn.model_selection import GridSearchCV
param_grid = {
            "n_estimators"      : [10,20,30],
            "max_features"      : ["auto", "sqrt", "log2"],
            "min_samples_split" : [2,4,8],
            "bootstrap": [True, False]
        }

grid = GridSearchCV(rf, param_grid, n_jobs =-1, cv=3)

In [14]:
# Fit the model using the grid search estimator. 
# This will take the SVC model and try each combination of parameters
grid.fit(X_train_scaled, y_train)

GridSearchCV(cv=3, estimator=RandomForestClassifier(n_estimators=200),
             n_jobs=-1,
             param_grid={'bootstrap': [True, False],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'min_samples_split': [2, 4, 8],
                         'n_estimators': [10, 20, 30]})

In [15]:
print(grid.best_params_)
print(grid.best_score_)

{'bootstrap': False, 'max_features': 'auto', 'min_samples_split': 8, 'n_estimators': 20}
0.8914724468272296


# Save the Model

In [16]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
randomForest_model= 'giangcao5.sav'
joblib.dump(randomForest_model, randomForest_model)

['giangcao5.sav']